In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os as os
import matplotlib.pyplot as plt

# own code base
import sys
sys.path.append("..")
import tf_loss_functions as lf
import splines as sp

## - Data Preparation

In [3]:
# Spline Hyperparameter

basis_dimension = 20
degree_bsplines = 3
penalty_diff_order = 2


zambia = pd.read_table('./data/zambia_height92.txt')

c_breastf_d = zambia.loc[:, 'c_breastf']
c_age_d = zambia.loc[:, "c_age"]
labels = zambia.loc[:, 'zscore']
labels_expanded = np.expand_dims(labels, 1)

# create splines from class
age = sp.pspline(x=c_age_d, degree_bsplines=degree_bsplines, penalty_diff_order=penalty_diff_order, knot_type="equi", basis_dimension=basis_dimension)
breastf = sp.pspline(x=c_breastf_d, degree_bsplines=degree_bsplines, penalty_diff_order=penalty_diff_order, knot_type="equi", basis_dimension=basis_dimension)


lambda_param_x1_est_values = []
lambda_param_x2_est_values = []
gcv_values = []
epochs_saved = []

## - Lambda Initialization: Grid Search

In [4]:
# Grid Search
lambda_param_range = np.log10(np.float32(np.array([1, 10, 100, 1000, 10000, 100000, 1000000])))
gcv_crit_values = []
lambda_param_x1_values = []
lambda_param_x2_values = []

for lambda_x1 in lambda_param_range:
    for lambda_x2 in lambda_param_range:
        value = lf.gcv_2d(
            y = labels_expanded, 
            design_matrix_Z_1 = age.design_matrix_d, 
            reg_matrix_K_1 = age.penalty_matrix_d, 
            reg_param_1 = np.exp(lambda_x1),
            design_matrix_Z_2 = breastf.design_matrix_d, 
            reg_matrix_K_2 = breastf.penalty_matrix_d, 
            reg_param_2 = np.exp(lambda_x2),
            )

        gcv_crit_values.append(value)
        lambda_param_x1_values.append(lambda_x1)
        lambda_param_x2_values.append(lambda_x2)

crit_values_min = np.min(gcv_crit_values)
lambda_param_x1_init = lambda_param_x1_values[np.argmin(gcv_crit_values)]
lambda_param_x2_init = lambda_param_x2_values[np.argmin(gcv_crit_values)]

# save starting points
epochs_saved.append(0)
lambda_param_x1_est_values.append(lambda_param_x1_init)
lambda_param_x2_est_values.append(lambda_param_x2_init)
gcv_values.append(crit_values_min)

2024-02-28 17:21:10.521762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


gcv: [[1.12214376e+08]]
gcv: [[1.2754976e+08]]
gcv: [[1.0080611e+08]]
gcv: [[32165928.]]
gcv: [[87273448.]]
gcv: [[59823416.]]
gcv: [[26300606.]]
gcv: [[718874.1]]
gcv: [[595675.94]]
gcv: [[1493650.]]
gcv: [[586085.]]
gcv: [[940342.]]
gcv: [[456756.78]]
gcv: [[264687.3]]
gcv: [[4.6780266e+08]]
gcv: [[3.0419658e+08]]
gcv: [[66753508.]]
gcv: [[5.441366e+08]]
gcv: [[77067520.]]
gcv: [[40725832.]]
gcv: [[48108828.]]
gcv: [[226744.12]]
gcv: [[167020.67]]
gcv: [[403570.2]]
gcv: [[363324.6]]
gcv: [[189385.97]]
gcv: [[58782.01]]
gcv: [[122840.09]]
gcv: [[574537.25]]
gcv: [[211684.14]]
gcv: [[104055.516]]
gcv: [[270954.9]]
gcv: [[331112.34]]
gcv: [[424399.72]]
gcv: [[129060.164]]
gcv: [[295042.25]]
gcv: [[480608.]]
gcv: [[857424.8]]
gcv: [[513638.]]
gcv: [[799060.75]]
gcv: [[214751.73]]
gcv: [[1067364.]]
gcv: [[271137.03]]
gcv: [[147792.92]]
gcv: [[143339.45]]
gcv: [[333552.4]]
gcv: [[157258.38]]
gcv: [[225282.62]]
gcv: [[187678.34]]


## - Model Building and Optimization

In [5]:
initializer = tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=13)

num_epochs_gcv = 5000 
num_epochs_splines = 1


lambda_param_x1 = tf.Variable(lambda_param_x1_init, name="lambda_x1", dtype=tf.float32, trainable=True)
lambda_param_x2 = tf.Variable(lambda_param_x2_init, name="lambda_x2", dtype=tf.float32, trainable=True)


weights_x1 = tf.Variable(initializer(shape=(basis_dimension, 1)), name="weights_x1")
weights_x2 = tf.Variable(initializer(shape=(basis_dimension, 1)), name="weights_x2")



def modelLoss(y, weights_x1, weights_x2, lambda_param_x1, lambda_param_x2, design_matrix_x1, penalty_matrix_x1, design_matrix_x2, penalty_matrix_x2):
    y_hat_x1 = tf.matmul(design_matrix_x1, weights_x1)
    y_hat_x2 = tf.matmul(design_matrix_x2, weights_x2)
    prediction = y_hat_x1 + y_hat_x2    
    sq_diff = tf.reduce_sum(input_tensor=(y - prediction) ** 2)
    regularizer_x1 = lambda_param_x1 * (tf.tensordot(tf.transpose(weights_x1), tf.tensordot(penalty_matrix_x1, weights_x1, axes = 1), axes = 1))
    regularizer_x2 = lambda_param_x2 * (tf.tensordot(tf.transpose(weights_x2), tf.tensordot(penalty_matrix_x2, weights_x2, axes = 1), axes = 1))
    loss = sq_diff + regularizer_x1 + regularizer_x2
    return loss


opt_splines = tf.keras.optimizers.Adam(learning_rate=0.1)
opt_gcv = tf.keras.optimizers.Adam(learning_rate=0.1)

## - Optimization Loop

In [6]:
for i in range(num_epochs_gcv):
    loss = lambda: lf.gcv_2d(
        y = labels_expanded, 
        design_matrix_Z_1 = age.design_matrix_d, 
        reg_matrix_K_1 = age.penalty_matrix_d, 
        reg_param_1 = tf.exp(lambda_param_x1),
        design_matrix_Z_2 = breastf.design_matrix_d, 
        reg_matrix_K_2 = breastf.penalty_matrix_d, 
        reg_param_2 = tf.exp(lambda_param_x2),
        )
                          
    opt_gcv.minimize(loss, var_list=[lambda_param_x1, lambda_param_x2])
    pen_loss = loss().numpy().item()

    for j in range(num_epochs_splines):
        loss = lambda: modelLoss(
            y=labels_expanded,
            weights_x1=weights_x1,
            weights_x2=weights_x2, 
            lambda_param_x1=tf.exp(lambda_param_x1), 
            lambda_param_x2=tf.exp(lambda_param_x2),
            design_matrix_x1=age.design_matrix_d,
            penalty_matrix_x1=age.penalty_matrix_d,  
            design_matrix_x2=breastf.design_matrix_d, 
            penalty_matrix_x2=breastf.penalty_matrix_d,
            )
        opt_splines.minimize(loss, var_list=[weights_x1, weights_x2]) 

    if (i+1)%100 == 0:
        print(f"Epoch: {i+1}") 
        epochs_saved.append(i+1)
        lambda_param_x1_est = tf.exp(lambda_param_x1).numpy()
        lambda_param_x2_est = tf.exp(lambda_param_x2).numpy()
        lambda_param_x1_est_values.append(lambda_param_x1_est)
        lambda_param_x2_est_values.append(lambda_param_x2_est)
        gcv_values.append(pen_loss)       
            
print(f"Final results: \n lambda_param_x1 = {tf.exp(lambda_param_x1).numpy()} \n lambda_param_x2 = {tf.exp(lambda_param_x2).numpy()}")

# save final results
epochs_saved.append(num_epochs_gcv)
lambda_param_x1_est = tf.exp(lambda_param_x1).numpy()
lambda_param_x2_est = tf.exp(lambda_param_x2).numpy()
lambda_param_x1_est_values.append(lambda_param_x1_est)
lambda_param_x2_est_values.append(lambda_param_x2_est)
gcv_values.append(pen_loss)



training_frame = pd.DataFrame(data=np.array([epochs_saved, lambda_param_x1_est_values, lambda_param_x2_est_values, gcv_values]).T, columns=["Epoch", "Lambda Parameter x1", "Lambda Parameter x2", "GCV"])
training_frame.to_csv(f"./results/smoothing_param_zambia_method=gcv_2d_epochs={num_epochs_gcv}.csv")

weights_x1_est = np.dot(age.U, weights_x1.numpy())
weights_x2_est = np.dot(breastf.U, weights_x2.numpy())

filename = f"./results/final_spline_zambia_method=gcv_2d_epoch={num_epochs_gcv}.npz"
np.savez(file = filename, reg_param_x1 = lambda_param_x1_est, reg_param_x2 = lambda_param_x2_est, weights_x1 = weights_x1_est, weights_x2 = weights_x2_est)

gcv: [[58782.01]]
gcv: [[801031.7]]
gcv: [[801031.7]]
gcv: [[1462892.1]]
gcv: [[1462892.1]]
gcv: [[139166.33]]
gcv: [[139166.33]]
gcv: [[874688.2]]
gcv: [[874688.2]]
gcv: [[1002135.7]]
gcv: [[1002135.7]]
gcv: [[3092998.5]]
gcv: [[3092998.5]]
gcv: [[795383.94]]
gcv: [[795383.94]]
gcv: [[1048062.56]]
gcv: [[1048062.56]]
gcv: [[220814.95]]
gcv: [[220814.95]]
gcv: [[6406884.]]
gcv: [[6406884.]]
gcv: [[164129.73]]
gcv: [[164129.73]]
gcv: [[104560.21]]
gcv: [[104560.21]]
gcv: [[1201260.6]]
gcv: [[1201260.6]]
gcv: [[3181019.]]
gcv: [[3181019.]]
gcv: [[430807.6]]
gcv: [[430807.6]]
gcv: [[927450.94]]
gcv: [[927450.94]]
gcv: [[8893034.]]
gcv: [[8893034.]]
gcv: [[33024.297]]
gcv: [[33024.297]]
gcv: [[185868.8]]
gcv: [[185868.8]]
gcv: [[1976012.1]]
gcv: [[1976012.1]]
gcv: [[119112.6]]
gcv: [[119112.6]]
gcv: [[2296843.2]]
gcv: [[2296843.2]]
gcv: [[73577.47]]
gcv: [[73577.47]]
gcv: [[13387939.]]
gcv: [[13387939.]]
gcv: [[3348461.]]
gcv: [[3348461.]]
gcv: [[56793.992]]
gcv: [[56793.992]]
gcv: [[14001